In [26]:
# 读txt文件
def readPatList(pat_file_list):
    ret_list_list=[]
    for i in pat_file_list:
        patlist = pd.read_table(i,sep='\t',header=None,index_col=None)[0].tolist()
        ret_list_list.append(patlist)
    
    ret_list =[]
    for i in ret_list_list:
        for j in i:
            ret_list.append(j)
    
    ret_list_ret = list(set(ret_list))
    
    print(f'读取 {len(pat_file_list)} 个文件,重合 {len(ret_list)-len(ret_list_ret)},共 {len(ret_list_ret)} 个病人')
    return ret_list_ret

# readPatList([r'../../分组/jsph_gbm.txt',r'../../分组/jsph_lgg.txt',
#                                      r'../../分组/TEST_jsph_gbm.txt',r'../../分组/TEST_jsph_lgg.txt'])

In [27]:
import pandas as pd
import numpy as np
import itertools
from scipy import stats
from tqdm import tqdm

# 打开影像组学特征

In [28]:
radiomics_orig = pd.read_csv('20_RiskScore_radiomics1402_pretreated_OS.csv',header=0,index_col=0,encoding='gbk')
radiomics_orig = radiomics_orig.T
radiomics_orig

,X0907236_ZhouJiSheng,X0875917_LiYan,TCGA-27-1830,TCGA-DU-5854,HF1588,TCGA-HT-8105,X0820029_YinYunHua,TCGA-HT-A61B,X1110489_ZhouGuiZhen,X0987375_ShiWeiDong,...,X0915255_ZhangWanSong,TCGA-06-0241,TCGA-DU-A5TU,X0954514_ChengNianXiang,TCGA-06-0185,X1052598_ShiHongMei,X0930877_XuPingLan,TCGA-DU-6395,HF1587,X1112145_XuZhangSong
status,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
time,6.830000,15.700000,5.130000,8.570000,74.200000,6.330000,8.030000,17.770000,12.300000,57.030000,...,72.600000,49.370000,26.530000,5.170000,74.870000,29.670000,8.630000,49.700000,74.200000,18.200000
RiskScore_h0,0.896298,0.901296,2.621098,0.627098,1.181956,1.982107,2.204183,0.355810,0.494902,0.249333,...,1.010866,1.538427,1.813689,0.571147,1.132816,0.619839,1.547634,2.314271,1.226855,1.096481
log.sigma.5.mm.3D_glcm_Idn,0.978929,0.976438,0.976809,0.975821,0.968646,0.983654,0.978770,0.978862,0.973409,0.968535,...,0.977288,0.969781,0.983478,0.977233,0.971411,0.973971,0.980352,0.977762,0.972784,0.982129
squareroot_firstorder_Median,418.103580,469.206017,361.326537,378.441206,389.782128,444.276436,479.962500,373.897167,453.421735,391.224457,...,479.649354,448.695440,389.620250,526.735661,393.688346,545.344729,496.023289,416.562123,447.538339,353.223980
lbp.3D.k_glrlm_ShortRunEmphasis,0.419132,0.372534,0.392058,0.297715,0.409873,0.437925,0.447526,0.437820,0.440920,0.400745,...,0.396581,0.270269,0.326767,0.395492,0.353708,0.405578,0.296317,0.339940,0.349452,0.412956
wavelet.LLH_glszm_GrayLevelNonUniformity,73.934508,48.381674,91.859655,13.598095,52.251438,138.985402,97.953928,104.142337,64.945720,29.380342,...,53.339339,22.960205,72.739010,77.866842,57.635891,66.154018,31.711602,437.999479,85.220925,45.477403
log.sigma.5.mm.3D_firstorder_90Percentile,13.716698,18.177860,-1.832864,9.866241,12.997869,26.281605,0.172801,-3.836824,-0.213360,4.360342,...,23.667278,-5.962297,16.099246,21.140311,14.404711,-6.453781,19.271812,4.845201,6.252650,11.987669
wavelet.LHH_ngtdm_Contrast,0.001638,0.000817,0.003141,0.001674,0.019593,0.000544,0.002292,0.003769,0.002393,0.007415,...,0.001274,0.061126,0.002829,0.020489,0.002263,0.001158,0.010592,0.002410,0.001351,0.003030
square_glrlm_RunEntropy,4.957925,4.973058,4.811214,4.827622,4.761555,5.091429,4.599873,4.831786,5.106352,5.047524,...,5.251952,4.777736,5.233897,5.101933,5.095664,5.066364,5.255231,4.611289,4.968237,5.273783


In [29]:
# 选择特征
radiomics = radiomics_orig.iloc[3:,:]
radiomics

,X0907236_ZhouJiSheng,X0875917_LiYan,TCGA-27-1830,TCGA-DU-5854,HF1588,TCGA-HT-8105,X0820029_YinYunHua,TCGA-HT-A61B,X1110489_ZhouGuiZhen,X0987375_ShiWeiDong,...,X0915255_ZhangWanSong,TCGA-06-0241,TCGA-DU-A5TU,X0954514_ChengNianXiang,TCGA-06-0185,X1052598_ShiHongMei,X0930877_XuPingLan,TCGA-DU-6395,HF1587,X1112145_XuZhangSong
log.sigma.5.mm.3D_glcm_Idn,0.978929,0.976438,0.976809,0.975821,0.968646,0.983654,0.978770,0.978862,0.973409,0.968535,...,0.977288,0.969781,0.983478,0.977233,0.971411,0.973971,0.980352,0.977762,0.972784,0.982129
squareroot_firstorder_Median,418.103580,469.206017,361.326537,378.441206,389.782128,444.276436,479.962500,373.897167,453.421735,391.224457,...,479.649354,448.695440,389.620250,526.735661,393.688346,545.344729,496.023289,416.562123,447.538339,353.223980
lbp.3D.k_glrlm_ShortRunEmphasis,0.419132,0.372534,0.392058,0.297715,0.409873,0.437925,0.447526,0.437820,0.440920,0.400745,...,0.396581,0.270269,0.326767,0.395492,0.353708,0.405578,0.296317,0.339940,0.349452,0.412956
wavelet.LLH_glszm_GrayLevelNonUniformity,73.934508,48.381674,91.859655,13.598095,52.251438,138.985402,97.953928,104.142337,64.945720,29.380342,...,53.339339,22.960205,72.739010,77.866842,57.635891,66.154018,31.711602,437.999479,85.220925,45.477403
log.sigma.5.mm.3D_firstorder_90Percentile,13.716698,18.177860,-1.832864,9.866241,12.997869,26.281605,0.172801,-3.836824,-0.213360,4.360342,...,23.667278,-5.962297,16.099246,21.140311,14.404711,-6.453781,19.271812,4.845201,6.252650,11.987669
wavelet.LHH_ngtdm_Contrast,0.001638,0.000817,0.003141,0.001674,0.019593,0.000544,0.002292,0.003769,0.002393,0.007415,...,0.001274,0.061126,0.002829,0.020489,0.002263,0.001158,0.010592,0.002410,0.001351,0.003030
square_glrlm_RunEntropy,4.957925,4.973058,4.811214,4.827622,4.761555,5.091429,4.599873,4.831786,5.106352,5.047524,...,5.251952,4.777736,5.233897,5.101933,5.095664,5.066364,5.255231,4.611289,4.968237,5.273783
wavelet.HLL_firstorder_Kurtosis,21.422334,15.557493,13.737185,22.605320,15.583998,18.803259,14.144041,17.304904,18.751277,38.595520,...,12.315222,9.126322,17.189388,10.239287,11.719906,32.544928,12.117864,20.043984,15.342903,25.800999
wavelet.HLL_glszm_SizeZoneNonUniformityNormalized,0.197382,0.136481,0.239300,0.159176,0.207264,0.193616,0.222305,0.248647,0.311124,0.202209,...,0.170925,0.131525,0.143627,0.171737,0.150845,0.263806,0.099305,0.413163,0.238279,0.246856
wavelet.LHL_glszm_ZoneEntropy,5.877973,7.213564,6.468888,5.824697,4.973512,6.316382,7.023200,6.257220,5.967760,6.368842,...,7.217918,6.605398,6.214094,6.888559,6.436709,6.738089,6.991156,4.848256,6.125211,6.423078


In [30]:
# 打开文件位置
jsph_lgg = readPatList([r'../../分组/jsph_lgg.txt'])
jsph_gbm = readPatList([r'../../分组/jsph_gbm.txt'])
TEST_jsph_lgg = readPatList([r'../../分组/TEST_jsph_lgg.txt'])
TEST_jsph_gbm = readPatList([r'../../分组/TEST_jsph_gbm.txt'])
jsph = readPatList([r'../../分组/jsph_lgg.txt',r'../../分组/jsph_gbm.txt',
                   r'../../分组/TEST_jsph_lgg.txt',r'../../分组/TEST_jsph_gbm.txt'])
 
rbrt_lgg = readPatList([r'../../分组/rbrt_lgg.txt'])
rbrt_gbm = readPatList([r'../../分组/rbrt_gbm.txt'])
rbrt = readPatList([r'../../分组/rbrt_lgg.txt',r'../../分组/rbrt_gbm.txt'])

tcga_lgg = readPatList([r'../../分组/tcga_lgg.txt'])
tcga_gbm = readPatList([r'../../分组/tcga_gbm.txt'])
tcga = readPatList([r'../../分组/tcga_lgg.txt',r'../../分组/tcga_gbm.txt'])

ALL = list(set(jsph)|set(rbrt)|set(tcga))
ALL_LGG = list(set(rbrt_lgg)|set(tcga_lgg)|set(jsph_lgg)|set(TEST_jsph_lgg))
ALL_GBM = list(set(rbrt_gbm)|set(tcga_gbm)|set(jsph_gbm)|set(TEST_jsph_gbm))

print(f'共合并到{len(ALL)}个病人,LGG{len(ALL_LGG)}个,GBM{len(ALL_GBM)}个')

读取 1 个文件,重合 0,共 84 个病人
读取 1 个文件,重合 0,共 85 个病人
读取 1 个文件,重合 0,共 93 个病人
读取 1 个文件,重合 0,共 77 个病人
读取 4 个文件,重合 0,共 339 个病人
读取 1 个文件,重合 0,共 50 个病人
读取 1 个文件,重合 0,共 23 个病人
读取 2 个文件,重合 0,共 73 个病人
读取 1 个文件,重合 0,共 134 个病人
读取 1 个文件,重合 0,共 117 个病人
读取 2 个文件,重合 0,共 251 个病人
共合并到663个病人,LGG361个,GBM302个


# 打开GSVA数据

In [31]:
"""
GSVA_file 输入原始未经筛选的GSVA结果
Pat_list 要筛选的病人列表

"""
def openGSVA_cross_toBeList(GSVA_file,Pat_list):
    # 读取待处理特征
    GSVA = pd.read_csv(GSVA_file,header=0,index_col=0)

    # 取影像组学数据和GSVA数据的交集
    GSVA_patlist = GSVA.columns.tolist()
    radiomics_patlist = radiomics.columns.tolist()
    patcross = list(set(GSVA_patlist) & set(radiomics_patlist))
    print(f'GSVA与影像组学交集 {len(patcross)} 个人')
    
    # 进一步获取想要的病人
    final_patcross = list(set(Pat_list) & set(patcross))
    print(f'最终能获取到 {len(final_patcross)} 人,比例{round(len(final_patcross)/len(patcross),3)*100}%')

    GSVA_cross = GSVA[final_patcross]
    radiomics_cross = radiomics[final_patcross]

    return([GSVA_cross,radiomics_cross])

# openGSVA_cross_toBeList(r'../→GSVA后数据/JSPH/GSVA_JSPH.csv',jsph)[1]

In [32]:
# 打开文件位置
GSVA_jsph = r'../→GSVA后数据/JSPH/GSVA_JSPH（没筛人）.csv'

GSVA_rbrt_lgg = r'../→GSVA后数据/RBRT_LGG/GSVA_RBRT_LGG（没筛人）.csv'
GSVA_rbrt_gbm = r'../→GSVA后数据/RBRT_GBM/GSVA_RBRT_GBM（没筛人）.csv'
GSVA_rbrt = r'../→GSVA后数据/RBRT/GSVA_RBRT（没筛人）.csv'

GSVA_tcga_lgg = r'../→GSVA后数据/TCGA_LGG/GSVA_TCGA_LGG（没筛人）.csv'
GSVA_tcga_gbm = r'../→GSVA后数据/TCGA_GBM/GSVA_TCGA_GBM（没筛人）.csv'
GSVA_tcga = r'../→GSVA后数据/TCGA/GSVA_TCGA（没筛人）.csv'

GSVA_ALL = r'../→GSVA后数据/ALL/GSVA_ALL（没筛人）.csv'
GSVA_ALL_LGG = r'../→GSVA后数据/ALL_LGG/GSVA_ALL_LGG（没筛人）.csv'
GSVA_ALL_GBM = r'../→GSVA后数据/ALL_GBM/GSVA_ALL_GBM（没筛人）.csv'

input_dict = {
    GSVA_jsph : openGSVA_cross_toBeList(GSVA_jsph,jsph),
    
    GSVA_rbrt_lgg : openGSVA_cross_toBeList(GSVA_rbrt_lgg,rbrt_lgg),
    GSVA_rbrt_gbm : openGSVA_cross_toBeList(GSVA_rbrt_gbm,rbrt_gbm),
    GSVA_rbrt : openGSVA_cross_toBeList(GSVA_rbrt,rbrt),
    
    GSVA_tcga_lgg : openGSVA_cross_toBeList(GSVA_tcga_lgg,tcga_lgg),
    GSVA_tcga_gbm : openGSVA_cross_toBeList(GSVA_tcga_gbm,tcga_gbm),
    GSVA_tcga : openGSVA_cross_toBeList(GSVA_tcga,tcga),
    
    GSVA_ALL : openGSVA_cross_toBeList(GSVA_ALL,ALL),
    GSVA_ALL_LGG : openGSVA_cross_toBeList(GSVA_ALL_LGG,ALL_LGG),
    GSVA_ALL_GBM : openGSVA_cross_toBeList(GSVA_ALL_GBM,ALL_GBM),
}

input_dict[GSVA_jsph][0]

GSVA与影像组学交集 13 个人
最终能获取到 13 人,比例100.0%
GSVA与影像组学交集 50 个人
最终能获取到 50 人,比例100.0%
GSVA与影像组学交集 24 个人
最终能获取到 23 人,比例95.8%
GSVA与影像组学交集 74 个人
最终能获取到 73 人,比例98.6%
GSVA与影像组学交集 126 个人
最终能获取到 126 人,比例100.0%
GSVA与影像组学交集 40 个人
最终能获取到 36 人,比例90.0%
GSVA与影像组学交集 166 个人
最终能获取到 165 人,比例99.4%
GSVA与影像组学交集 253 个人
最终能获取到 251 人,比例99.2%
GSVA与影像组学交集 176 个人
最终能获取到 176 人,比例100.0%
GSVA与影像组学交集 77 个人
最终能获取到 71 人,比例92.2%


,X1153627_ZhangTaiXiang,X1156175_LiXia,X1164833_DongXiangZheng,X1010577_LiXiuJin,X1156065_ZhangDaChang,X1137100_DuHuiBin,X1162807_YuanZuYou,X1031574_LiLinSheng,X1029842_XuLongSheng,X1002249_GuCongXiang,X1027770_LiuYueQin,X1151106_YuGuoPing,X1163024_XuMeiFang
GO_ID,,,,,,,,,,,,,
GO:0006068,0.064799,0.103372,0.047452,0.091104,0.066458,0.119238,0.083577,0.092499,0.049614,0.029023,0.126391,0.101321,0.051738
GO:0035767,0.135060,0.125742,0.220400,0.131306,0.223113,0.160428,0.085666,0.120412,0.126323,0.131524,0.131787,0.104882,0.223273
GO:0050961,-0.014986,-0.033857,-0.052655,-0.018792,0.019042,0.014265,0.009211,0.024710,0.013616,-0.003892,0.003351,-0.044768,0.013160
GO:0051014,0.278672,0.179922,0.261676,0.247812,0.285707,0.278392,0.253227,0.238759,0.283304,0.212991,0.270715,0.259505,0.314108
GO:0097205,0.069793,0.016833,0.124049,0.119375,0.165628,0.130989,0.050569,0.083441,0.110088,0.075210,0.125426,0.071983,0.147059
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0002269,0.133978,0.084523,0.113942,0.147463,0.184923,0.170140,0.123223,0.153243,0.166677,0.166544,0.166143,0.142848,0.181908
GO:0006195,0.172320,0.114349,0.139983,0.160451,0.102196,0.142345,0.179344,0.153304,0.113694,0.152509,0.135152,0.149278,0.129553
GO:0005092,0.334752,0.297045,0.277791,0.316957,0.272996,0.269365,0.326293,0.320826,0.287371,0.305288,0.297827,0.333714,0.301757


# 做相关：二维表

In [33]:
def make_two_corrTable(GSVA_cross,radiomics_cross,open_file_name):
    x = list(radiomics_cross.index)  # 获取最终表的列索引
    y = list(GSVA_cross.index)
    retDic_cor = dict([(k,[]) for k in x])
    retDic_p = dict([(k,[]) for k in x])

    method = 'pearson' # method : {'pearson', 'spearman', 'kendall'} # 选择⽅法

    for i in tqdm(range(len(x))):
        fea = x[i]
        for rn in y:
            varSet1 = radiomics_cross.loc[fea].values
            varSet2 = GSVA_cross.loc[rn].values

            # 选择⽅法进⾏计算
            if method == 'pearson':
                r, p = stats.pearsonr(varSet1, varSet2)
            elif method == 'spearman':
                r, p = stats.spearmanr(varSet1, varSet2, nan_policy='omit')
            elif method == 'kendall':
                r, p = stats.kendalltau(varSet1, varSet2, nan_policy='omit')
            else:
                print("method⽅法选择错误")


            # 向相关度表中添加内容
            retDic_cor[fea].append(r)
            # 向p值表中添加内容
            retDic_p[fea].append(p)

    df_cor = pd.DataFrame(data=retDic_cor,index=GSVA_cross.index,columns=x)
    df_p = pd.DataFrame(data=retDic_p,index=GSVA_cross.index,columns=x)
    
    save_name1 = (key[:-4]+'_二维_相关系数表.csv').replace('（没筛人）','')
    save_name2 = (key[:-4]+'_二维_可信度表.csv').replace('（没筛人）','')

    df_cor.to_csv(save_name1, index=True)
    df_p.to_csv(save_name2, index=True)

# 最终运行

In [35]:
for key,value in input_dict.items():
    print(key)
#     print((key[:-4]+'_二维_相关系数表.csv').replace('（没筛人）',''))
    
    make_two_corrTable(value[0],value[1],key)

../→GSVA后数据/JSPH/GSVA_JSPH（没筛人）.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.59it/s]


../→GSVA后数据/RBRT_LGG/GSVA_RBRT_LGG（没筛人）.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.64it/s]


../→GSVA后数据/RBRT_GBM/GSVA_RBRT_GBM（没筛人）.csv


...

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.61it/s]


../→GSVA后数据/ALL_LGG/GSVA_ALL_LGG（没筛人）.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.62it/s]


../→GSVA后数据/ALL_GBM/GSVA_ALL_GBM（没筛人）.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.63it/s]
